# Laporan Proyek Machine Learning - SistemRekomendasi

## Pendahuluan

Sistem rekomendasi daur ulang sampah dirancang untuk membantu pengguna dalam mengelola sampah dengan memberikan saran daur ulang yang sesuai berdasarkan jenis sampah dan beratnya. Sistem ini menggunakan pendekatan berbasis teks dengan TF-IDF Vectorizer dan cosine similarity untuk mencocokkan deskripsi sampah pengguna dengan rekomendasi dalam dataset, serta mempertimbangkan rentang berat sampah dalam satuan kilogram. Tujuan utama adalah mempromosikan pengelolaan sampah yang ramah lingkungan dengan rekomendasi yang praktis dan spesifik.

## Metodologi

Dataset

Dataset digunakan dalam format JSON (dataset_rekomendasi_daur_ulang_berat_lebih_lebar.json) dan berisi beberapa entri untuk setiap kategori sampah, seperti Kardus, Plastik, dan Vegetasi. Setiap entri memiliki:

1. `kategori`: Nama kategori sampah (misalnya, Plastik, Kertas).
2. `berat_min_kg`: Berat minimum dalam kilogram.
3. `berat_max_kg`: Berat maksimum dalam kilogram.

rekomendasi: Daftar saran daur ulang yang relevan dengan rentang berat.

Contoh struktur dataset:

```[
    {
        "kategori": "Plastik",
        "berat_min_kg": 0.01,
        "berat_max_kg": 0.5,
        "rekomendasi": [
            "Cuci kantong plastik atau botol kecil sebelum didaur ulang.",
            "Gunakan kembali kantong plastik untuk keperluan rumah tangga.",
            "Buang plastik ringan ke tempat pengumpulan plastik di bank sampah."
        ]
    },
    .....
]

In [1]:
import json
import pandas as pd
import tensorflow as tf
import numpy as np

2025-05-01 00:00:57.653481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746032458.271365   65520 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746032458.429017   65520 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746032459.744919   65520 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746032459.744947   65520 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746032459.744950   65520 computation_placer.cc:177] computation placer alr

In [2]:
# Load Dataset
try:
    with open("../data/rekomendasi/dataset_rekomendasi_daur_ulang.json", "r", encoding="utf-8") as file:
        data = json.load(file)
except FileNotFoundError:
    print("Error: File dataset tidak ditemukan.")
    exit(1)
except json.JSONDecodeError:
    print("Error: Format JSON tidak valid.")
    exit(1)

# Konversi data menjadi DataFrame
df = pd.DataFrame(data)

# Gabungkan rekomendasi menjadi satu kolom untuk vektorisasi
df['combined_rekomendasi'] = df['rekomendasi'].apply(lambda x: " ".join(x))

In [3]:
df.head()

,kategori,berat_min_kg,berat_max_kg,rekomendasi,combined_rekomendasi
0,Kardus,0.10,1.0,[Lipat kardus kecil seperti kotak sepatu untuk...,Lipat kardus kecil seperti kotak sepatu untuk ...
1,Kardus,1.00,10.0,[Potong kardus besar menjadi bagian kecil untu...,Potong kardus besar menjadi bagian kecil untuk...
2,Kardus,10.00,30.0,[Jual ke pengepul kardus untuk daur ulang skal...,Jual ke pengepul kardus untuk daur ulang skala...
3,Kardus,30.00,100.0,[Hubungi pabrik daur ulang kertas untuk pengam...,Hubungi pabrik daur ulang kertas untuk pengamb...
4,Bahan Organik Makanan,0.05,1.0,[Kompos sisa makanan harian seperti kulit sayu...,Kompos sisa makanan harian seperti kulit sayur...


## Pre-Processing

In [4]:
texts = df['combined_rekomendasi'].values  # Teks rekomendasi
labels = np.arange(len(texts))  # Indeks sebagai label (untuk mencocokkan entri)

# lapisan vektorisasi teks
max_tokens = 1000  # Jumlah kata unik maksimum

text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode='tf_idf'  # Menggunakan TF-IDF
)

# Adapt vectorizer ke data teks
text_vectorizer.adapt(texts)

I0000 00:00:1746032530.683640   65520 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2249 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


## modelling

In [5]:
inputs = tf.keras.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
outputs = x  # Output adalah vektor TF-IDF

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss='mse')  # Loss tidak terlalu relevan karena ini bukan pelatihan supervised

# 5. Fungsi untuk mendapatkan rekomendasi
def get_recommendation(model, input_kategori, input_berat_kg, df, tolerance=0.2):
    # Cari kategori dalam dataset untuk kecocokan teks
    matching_rows = df[df['kategori'] == input_kategori]
    if matching_rows.empty:
        return {"message": "Kategori tidak ditemukan dalam dataset."}

    # Vektorisasi input teks (kategori) - pastikan input kategori adalah string, bukan list
    # FIX: Make sure input_kategori is passed as a single element array of strings
    input_vec = model.predict(np.array([input_kategori]))[0]
    
    # Vektorisasi semua teks rekomendasi dalam dataset
    # FIX: Convert to numpy array to ensure consistent handling
    rekomendasi_texts = matching_rows['combined_rekomendasi'].values
    dataset_vecs = model.predict(np.array(rekomendasi_texts))
    
    # Hitung cosine similarity
    cosine_sim = np.dot(dataset_vecs, input_vec) / (np.linalg.norm(dataset_vecs, axis=1) * np.linalg.norm(input_vec))
    
    # Ambil indeks dengan skor tertinggi
    best_match_idx = np.argmax(cosine_sim)
    
    # Dapatkan data dari entri terbaik
    kategori = matching_rows.iloc[best_match_idx]['kategori']
    rekomendasi_list = matching_rows.iloc[best_match_idx]['rekomendasi']
    berat_min_kg = matching_rows.iloc[best_match_idx]['berat_min_kg']
    berat_max_kg = matching_rows.iloc[best_match_idx]['berat_max_kg']
    
    # Periksa rentang berat
    min_tolerant = berat_min_kg * (1 - tolerance)
    max_tolerant = berat_max_kg * (1 + tolerance)
    
    if min_tolerant <= input_berat_kg <= max_tolerant:
        message = ""
    else:
        message = (
            f"Berat sampah Anda ({input_berat_kg} kg) sedikit tidak sesuai dengan rekomendasi "
            f"untuk kategori ini ({berat_min_kg} kg - {berat_max_kg} kg).\n\n"
            "Namun, berikut adalah beberapa rekomendasi yang bisa diterapkan:\n"
        )
    
    return {
        "kategori": kategori,
        "berat_input_kg": input_berat_kg,
        "berat_min_kg": berat_min_kg,
        "berat_max_kg": berat_max_kg,
        "message": message,
        "rekomendasi": rekomendasi_list
    }


## Save Model

In [6]:
# 6. Simpan model
model.save("../models/Sistemrekomendasi/models/recycling_recommendation_model.h5")
# Simpan dalam format SavedModel
print("Model berhasil disimpan di folder 'Sistemrekomendasi/models/recycling_recommendation_model'")

# 7. Simpan dataset untuk penggunaan BE/FE
df.to_json("../models/Sistemrekomendasi/dataset.json", orient="records", lines=False, force_ascii=False)
print("Dataset disimpan sebagai 'dataset.json'")

Model berhasil disimpan di folder 'Sistemrekomendasi/models/recycling_recommendation_model'
Dataset disimpan sebagai 'dataset.json'


### Testing

In [16]:
import json
import pandas as pd
import tensorflow as tf
import numpy as np
import os

def test_recycling_system():
    print("======= TESTING SISTEM REKOMENDASI DAUR ULANG =======")
    
    # 1. Load Dataset
    try:
        dataset_path = "../models/Sistemrekomendasi/dataset.json"
        if not os.path.exists(dataset_path):
            print(f"Dataset tidak ditemukan di path: {dataset_path}")
            print("Mencoba mencari di direktori lokal...")
            dataset_path = "dataset.json"
            
            if not os.path.exists(dataset_path):
                dataset_path = "../dataset_rekomendasi_daur_ulang.json"
                if not os.path.exists(dataset_path):
                    print("Error: Dataset tidak ditemukan. Coba mencari dataset asli...")
                    try:
                        with open("../dataset_rekomendasi_daur_ulang.json", "r", encoding="utf-8") as file:
                            data = json.load(file)
                            df = pd.DataFrame(data)
                            print("Dataset asli berhasil dimuat.")
                    except FileNotFoundError:
                        print("Error: Semua upaya untuk menemukan dataset gagal.")
                        print("Silakan sesuaikan path dataset terlebih dahulu!")
                        return
                    except json.JSONDecodeError:
                        print("Error: Format JSON tidak valid.")
                        return
                else:
                    print(f"Dataset ditemukan di: {dataset_path}")
                    df = pd.read_json(dataset_path)
            else:
                print(f"Dataset ditemukan di direktori lokal")
                df = pd.read_json(dataset_path)
        else:
            print(f"Dataset ditemukan di: {dataset_path}")
            df = pd.read_json(dataset_path)
            
        print(f"Total kategori dalam dataset: {len(df['kategori'].unique())}")
        print(f"Kategori sampah tersedia: {', '.join(df['kategori'].unique())}")
        
    except Exception as e:
        print(f"Error saat memuat dataset: {str(e)}")
        return
    
    # Cek apakah kolom combined_rekomendasi sudah ada
    if 'combined_rekomendasi' not in df.columns:
        df['combined_rekomendasi'] = df['rekomendasi'].apply(lambda x: " ".join(x))


    # 2. Load model yang sudah disimpan
    print("\nMemuat model vektorisasi teks yang telah disimpan...")
    model_path = "../models/Sistemrekomendasi/models/recycling_recommendation_model.h5"
    if not os.path.exists(model_path):
        print(f"Error: Model tidak ditemukan di {model_path}")
        return
    model = tf.keras.models.load_model(model_path, compile=False)
    print("Model berhasil dimuat.")

    # 3. Fungsi untuk mendapatkan rekomendasi
    def get_recommendation(model, input_kategori, input_berat_kg, df, tolerance=0.2):
        matching_rows = df[df['kategori'] == input_kategori]
        if matching_rows.empty:
            return {"message": f"Kategori '{input_kategori}' tidak ditemukan dalam dataset."}
    
        # Cari indeks baris yang beratnya sesuai
        weight_match = matching_rows[
            (matching_rows['berat_min_kg'] <= input_berat_kg) &
            (input_berat_kg <= matching_rows['berat_max_kg'])
        ]
    
        if not weight_match.empty:
            best_match_idx = weight_match.index[0]  # Ambil indeks baris yang sesuai
            message = ""
        else:
            best_match_idx = matching_rows.index[0]  # Fallback ke baris pertama
            berat_min_kg = matching_rows.loc[best_match_idx]['berat_min_kg']
            berat_max_kg = matching_rows.loc[best_match_idx]['berat_max_kg']
            message = (
                f"Berat sampah Anda ({input_berat_kg} kg) sedikit tidak sesuai dengan rekomendasi "
                f"({berat_min_kg} kg - {berat_max_kg} kg).\n\n"
                "Namun, berikut adalah beberapa rekomendasi yang bisa diterapkan:\n"
            )
    
        kategori = matching_rows.loc[best_match_idx]['kategori']
        rekomendasi_list = matching_rows.loc[best_match_idx]['rekomendasi']
        berat_min_kg = matching_rows.loc[best_match_idx]['berat_min_kg']
        berat_max_kg = matching_rows.loc[best_match_idx]['berat_max_kg']
    
        return {
            "kategori": kategori,
            "berat_input_kg": input_berat_kg,
            "berat_min_kg": berat_min_kg,
            "berat_max_kg": berat_max_kg,
            "message": message,
            "rekomendasi": rekomendasi_list
        }
    

    # 4. Jalankan test cases
    def run_test_cases(model, df):
        available_categories = df['kategori'].unique().tolist()
        
        test_cases = []
        for cat in available_categories:
            cat_rows = df[df['kategori'] == cat]
            if not cat_rows.empty:
                min_weight = cat_rows['berat_min_kg'].values[0]
                max_weight = cat_rows['berat_max_kg'].values[0]
                
                in_range_weight = (min_weight + max_weight) / 2
                test_cases.append({
                    "kategori": cat, 
                    "berat": in_range_weight, 
                    "desc": f"{cat} dengan berat dalam rentang"
                })
                
                out_range_weight = max_weight * 2
                test_cases.append({
                    "kategori": cat, 
                    "berat": out_range_weight, 
                    "desc": f"{cat} dengan berat di luar rentang"
                })
        
        test_cases.append({
            "kategori": "TidakAda", 
            "berat": 1.0, 
            "desc": "Kategori yang tidak ada di dataset"
        })
        
        print("\n" + "=" * 50)
        print("HASIL PENGUJIAN DENGAN BERBAGAI KASUS")
        print("=" * 50)
        
        for i, test in enumerate(test_cases, 1):
            print(f"\nTest Case #{i}: {test['desc']}")
            print("-" * 50)
            
            result = get_recommendation(model, test['kategori'], test['berat'], df)
            
            if "message" in result and "kategori" not in result:
                print(result["message"])
                continue
                
            print(f"Kategori Sampah: {result['kategori']}")
            print(f"Berat Sampah: {result['berat_input_kg']} kg (Rentang: {result['berat_min_kg']} kg - {result['berat_max_kg']} kg)")
            
            if result['message']:
                print(f"\n{result['message']}")
                
            print("Rekomendasi:")
            for idx, ide in enumerate(result['rekomendasi'], 1):
                print(f"{idx}. {ide}")

    # 5. Uji sistem interaktif
    def interactive_test():
        print("\n" + "=" * 50)
        print("PENGUJIAN INTERAKTIF")
        print("=" * 50)
        print("Ketik 'exit' untuk keluar dari pengujian interaktif.")
        print(f"Kategori tersedia: {', '.join(df['kategori'].unique())}")
        
        while True:
            kategori = input("\nMasukkan kategori sampah: ")
            if kategori.lower() == 'exit':
                break
                
            try:
                berat = float(input("Masukkan berat sampah dalam kg: "))
            except ValueError:
                print("Error: Berat harus berupa angka. Silakan coba lagi.")
                continue
                
            result = get_recommendation(model, kategori, berat, df)
            
            if "message" in result and "kategori" not in result:
                print(result["message"])
                continue
                
            print("\nHasil Rekomendasi:")
            print(f"Kategori Sampah: {result['kategori']}")
            print(f"Berat Sampah: {result['berat_input_kg']} kg (Rentang: {result['berat_min_kg']} kg - {result['berat_max_kg']} kg)")
            
            if result['message']:
                print(f"\n{result['message']}")
                
            print("Rekomendasi:")
            for idx, ide in enumerate(result['rekomendasi'], 1):
                print(f"{idx}. {ide}")
    
    # Jalankan semua pengujian
    run_test_cases(model, df)
    
    try_interactive = input("\nApakah Anda ingin mencoba pengujian interaktif? (y/n): ")
    if try_interactive.lower() == 'y':
        interactive_test()
    
    print("\n======= PENGUJIAN SELESAI =======")

# Jalankan fungsi
if __name__ == "__main__":
    test_recycling_system()


======= TESTING SISTEM REKOMENDASI DAUR ULANG =======
Dataset ditemukan di: ../models/Sistemrekomendasi/dataset.json
Total kategori dalam dataset: 9
Kategori sampah tersedia: Kardus, Bahan Organik Makanan, Kaca, Logam, Sampah Lainnya, Kertas, Plastik, Sampah Tekstil, Vegetasi

Memuat model vektorisasi teks yang telah disimpan...
Model berhasil dimuat.

HASIL PENGUJIAN DENGAN BERBAGAI KASUS

Test Case #1: Kardus dengan berat dalam rentang
--------------------------------------------------
Kategori Sampah: Kardus
Berat Sampah: 0.55 kg (Rentang: 0.1 kg - 1.0 kg)
Rekomendasi:
1. Lipat kardus kecil seperti kotak sepatu untuk menghemat ruang.
2. Gunakan kardus untuk kerajinan tangan atau proyek sekolah.
3. Daur ulang di bank sampah atau tempat pengumpulan kertas.

Test Case #2: Kardus dengan berat di luar rentang
--------------------------------------------------
Kategori Sampah: Kardus
Berat Sampah: 2.0 kg (Rentang: 1.0 kg - 10.0 kg)
Rekomendasi:
1. Potong kardus besar menjadi bagian kecil 

### Testing 2

In [14]:
import json
import pandas as pd
import tensorflow as tf
import numpy as np
import os

def test_recycling_system():
    print("======= TESTING SISTEM REKOMENDASI DAUR ULANG =======")
    
    # 1. Load Dataset
    try:
        dataset_path = "../models/Sistemrekomendasi/dataset.json"
        if not os.path.exists(dataset_path):
            print(f"Dataset tidak ditemukan di path: {dataset_path}")
            print("Mencoba mencari di direktori lokal...")
            dataset_path = "dataset.json"
            
            if not os.path.exists(dataset_path):
                dataset_path = "../dataset_rekomendasi_daur_ulang.json"
                if not os.path.exists(dataset_path):
                    print("Error: Dataset tidak ditemukan. Coba mencari dataset asli...")
                    try:
                        with open("../dataset_rekomendasi_daur_ulang.json", "r", encoding="utf-8") as file:
                            data = json.load(file)
                            df = pd.DataFrame(data)
                            print("Dataset asli berhasil dimuat.")
                    except FileNotFoundError:
                        print("Error: Semua upaya untuk menemukan dataset gagal.")
                        print("Silakan sesuaikan path dataset terlebih dahulu!")
                        return
                    except json.JSONDecodeError:
                        print("Error: Format JSON tidak valid.")
                        return
                else:
                    print(f"Dataset ditemukan di: {dataset_path}")
                    df = pd.read_json(dataset_path)
            else:
                print(f"Dataset ditemukan di direktori lokal")
                df = pd.read_json(dataset_path)
        else:
            print(f"Dataset ditemukan di: {dataset_path}")
            df = pd.read_json(dataset_path)
            
        print(f"Total kategori dalam dataset: {len(df['kategori'].unique())}")
        print(f"Kategori sampah tersedia: {', '.join(df['kategori'].unique())}")
        
    except Exception as e:
        print(f"Error saat memuat dataset: {str(e)}")
        return
    
    # Cek apakah kolom combined_rekomendasi sudah ada
    if 'combined_rekomendasi' not in df.columns:
        df['combined_rekomendasi'] = df['rekomendasi'].apply(lambda x: " ".join(x))


    # 2. Load model yang sudah disimpan
    print("\nMemuat model vektorisasi teks yang telah disimpan...")
    model_path = "../models/Sistemrekomendasi/models/recycling_recommendation_model.h5"
    if not os.path.exists(model_path):
        print(f"Error: Model tidak ditemukan di {model_path}")
        return
    model = tf.keras.models.load_model(model_path, compile=False)
    print("Model berhasil dimuat.")

    # 3. Fungsi untuk mendapatkan rekomendasi
    def get_recommendation(model, input_kategori, input_berat_kg, df, tolerance=0.2):
        matching_rows = df[df['kategori'] == input_kategori]
        if matching_rows.empty:
            return {"message": f"Kategori '{input_kategori}' tidak ditemukan dalam dataset."}
    
        # Cari indeks baris yang beratnya sesuai
        weight_match = matching_rows[
            (matching_rows['berat_min_kg'] <= input_berat_kg) &
            (input_berat_kg <= matching_rows['berat_max_kg'])
        ]
    
        if not weight_match.empty:
            best_match_idx = weight_match.index[0]  # Ambil indeks baris yang sesuai
            message = ""
        else:
            best_match_idx = matching_rows.index[0]  # Fallback ke baris pertama
            berat_min_kg = matching_rows.loc[best_match_idx]['berat_min_kg']
            berat_max_kg = matching_rows.loc[best_match_idx]['berat_max_kg']
            message = (
                f"Berat sampah Anda ({input_berat_kg} kg) sedikit tidak sesuai dengan rekomendasi "
                f"({berat_min_kg} kg - {berat_max_kg} kg).\n\n"
                "Namun, berikut adalah beberapa rekomendasi yang bisa diterapkan:\n"
            )
    
        kategori = matching_rows.loc[best_match_idx]['kategori']
        rekomendasi_list = matching_rows.loc[best_match_idx]['rekomendasi']
        berat_min_kg = matching_rows.loc[best_match_idx]['berat_min_kg']
        berat_max_kg = matching_rows.loc[best_match_idx]['berat_max_kg']
    
        return {
            "kategori": kategori,
            "berat_input_kg": input_berat_kg,
            "berat_min_kg": berat_min_kg,
            "berat_max_kg": berat_max_kg,
            "message": message,
            "rekomendasi": rekomendasi_list
        }
    

    # 4. Jalankan test cases
    def run_test_cases(model, df):
        available_categories = df['kategori'].unique().tolist()
        
        test_cases = []
        for cat in available_categories:
            cat_rows = df[df['kategori'] == cat]
            if not cat_rows.empty:
                min_weight = cat_rows['berat_min_kg'].values[0]
                max_weight = cat_rows['berat_max_kg'].values[0]
                
                in_range_weight = (min_weight + max_weight) / 2
                test_cases.append({
                    "kategori": cat, 
                    "berat": in_range_weight, 
                    "desc": f"{cat} dengan berat dalam rentang"
                })
                
                out_range_weight = max_weight * 2
                test_cases.append({
                    "kategori": cat, 
                    "berat": out_range_weight, 
                    "desc": f"{cat} dengan berat di luar rentang"
                })
        
        test_cases.append({
            "kategori": "TidakAda", 
            "berat": 1.0, 
            "desc": "Kategori yang tidak ada di dataset"
        })
        
        print("\n" + "=" * 50)
        print("HASIL PENGUJIAN DENGAN BERBAGAI KASUS")
        print("=" * 50)
        
        for i, test in enumerate(test_cases, 1):
            print(f"\nTest Case #{i}: {test['desc']}")
            print("-" * 50)
            
            result = get_recommendation(model, test['kategori'], test['berat'], df)
            
            if "message" in result and "kategori" not in result:
                print(result["message"])
                continue
                
            print(f"Kategori Sampah: {result['kategori']}")
            print(f"Berat Sampah: {result['berat_input_kg']} kg (Rentang: {result['berat_min_kg']} kg - {result['berat_max_kg']} kg)")
            
            if result['message']:
                print(f"\n{result['message']}")
                
            print("Rekomendasi:")
            for idx, ide in enumerate(result['rekomendasi'], 1):
                print(f"{idx}. {ide}")

    # 5. Uji sistem interaktif
    def interactive_test():
        print("\n" + "=" * 50)
        print("PENGUJIAN INTERAKTIF")
        print("=" * 50)
        print("Ketik 'exit' untuk keluar dari pengujian interaktif.")
        print(f"Kategori tersedia: {', '.join(df['kategori'].unique())}")
        
        while True:
            kategori = input("\nMasukkan kategori sampah: ")
            if kategori.lower() == 'exit':
                break
                
            try:
                berat = float(input("Masukkan berat sampah dalam kg: "))
            except ValueError:
                print("Error: Berat harus berupa angka. Silakan coba lagi.")
                continue
                
            result = get_recommendation(model, kategori, berat, df)
            
            if "message" in result and "kategori" not in result:
                print(result["message"])
                continue
                
            print("\nHasil Rekomendasi:")
            print(f"Kategori Sampah: {result['kategori']}")
            print(f"Berat Sampah: {result['berat_input_kg']} kg (Rentang: {result['berat_min_kg']} kg - {result['berat_max_kg']} kg)")
            
            if result['message']:
                print(f"\n{result['message']}")
                
            print("Rekomendasi:")
            for idx, ide in enumerate(result['rekomendasi'], 1):
                print(f"{idx}. {ide}")
    
    # Jalankan semua pengujian
    run_test_cases(model, df)
    
    try_interactive = input("\nApakah Anda ingin mencoba pengujian interaktif? (y/n): ")
    if try_interactive.lower() == 'y':
        interactive_test()
    
    print("\n======= PENGUJIAN SELESAI =======")

# Jalankan fungsi
if __name__ == "__main__":
    test_recycling_system()


======= TESTING SISTEM REKOMENDASI DAUR ULANG =======
Dataset ditemukan di: ../models/Sistemrekomendasi/dataset.json
Total kategori dalam dataset: 9
Kategori sampah tersedia: Kardus, Bahan Organik Makanan, Kaca, Logam, Sampah Lainnya, Kertas, Plastik, Sampah Tekstil, Vegetasi

Memuat model vektorisasi teks yang telah disimpan...
Model berhasil dimuat.

HASIL PENGUJIAN DENGAN BERBAGAI KASUS

Test Case #1: Kardus dengan berat dalam rentang
--------------------------------------------------
Kategori Sampah: Kardus
Berat Sampah: 0.55 kg (Rentang: 0.1 kg - 1.0 kg)
Rekomendasi:
1. Lipat kardus kecil seperti kotak sepatu untuk menghemat ruang.
2. Gunakan kardus untuk kerajinan tangan atau proyek sekolah.
3. Daur ulang di bank sampah atau tempat pengumpulan kertas.

Test Case #2: Kardus dengan berat di luar rentang
--------------------------------------------------
Kategori Sampah: Kardus
Berat Sampah: 2.0 kg (Rentang: 1.0 kg - 10.0 kg)
Rekomendasi:
1. Potong kardus besar menjadi bagian kecil 